**Load the required libraries and data**

In [1]:
# Libraries 
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns

In [2]:
# Data  
demographics = pd.read_csv("../data/Demographic Data.csv", 
                             encoding='UTF-16', delimiter="\t")

# EXPLANATORY DATA ANALYSIS 
## Data Assessment

In [3]:
demographics.head()

,Chart#,Ethnicity,Race,Gender,Address,City,State,Zipcode,Intake Education Degree,Intake Employment Current Employer,...,Intake Employment Retired,Intake Employment Student,Intake Employment Unemployed,Intake Employment Working,Intake Family Divorced,Intake Family Married,Intake Family Partnered,Intake Family Single,Intake Family Widowed,Unnamed: 21
0,NaN,,,NaN,NaN,NaN,NaN,NaN,,,...,,,,,,,,,,NaN
1,1.0,,,F,253 NORFOLK STREET,CAMBRIDGE,MA,02139,,,...,,,,,,,,,,NaN
2,2.0,,,F,48 UNION PARK,BOSTON,MA,02118,,,...,,,,,,,,,,NaN
3,3.0,,,M,164 PALMER STREET,ARLINGTON,MA,02474,,,...,,,,,,,,,,NaN
4,4.0,,,F,110 HARTWELL ROAD,BEDFORD,MA,01730,,,...,,,,,,,,,,NaN


In [4]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686 entries, 0 to 2685
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Chart#                              2685 non-null   float64
 1   Ethnicity                           2686 non-null   object 
 2   Race                                2686 non-null   object 
 3   Gender                              2685 non-null   object 
 4   Address                             2666 non-null   object 
 5   City                                2663 non-null   object 
 6   State                               2661 non-null   object 
 7   Zipcode                             2665 non-null   object 
 8   Intake Education Degree             2684 non-null   object 
 9   Intake Employment Current Employer  2580 non-null   object 
 10  Intake Employment Disabled          2412 non-null   object 
 11  Intake Employment Occupation        2632 no

In [5]:
demographics["Ethnicity"].unique()

array([' ', 'Not Hispanic or Latino', 'Hispanic or Latino',
       'Declined to Specify', 'Unknown'], dtype=object)

In [9]:
demographics["Chart#"].nunique()

2685

## Data Wrangling 

## Data Analysis and Visualization

In [6]:
demographics["Ethnicity"].value_counts()

Ethnicity
                          2636
Not Hispanic or Latino      41
Hispanic or Latino           4
Declined to Specify          4
Unknown                      1
Name: count, dtype: int64

In [7]:
demographics["Race"].value_counts()

Race
                                             1585
White                                         842
Declined to Specify                           131
Asian                                          36
Black or African American                      11
Other Race                                      9
Chinese                                         7
Asian Indian                                    6
European                                        5
Irish                                           5
Korean                                          5
Scottish                                        5
Middle Eastern or North African                 4
German                                          3
Taiwanese                                       3
Italian                                         3
Native Hawaiian or Other Pacific Islander       2
Polish                                          2
American Indian or Alaska Native                2
Armenian                                     